In [1]:
import numpy as np
import pandas as pd 
from datetime import datetime

%matplotlib inline
import matplotlib
%config InlineBackend.figure_format = 'jpg'
matplotlib.figure.Figure.__repr__ = lambda self: (
    f"<{self.__class__.__name__} size {self.bbox.size[0]:g}"
    f"x{self.bbox.size[1]:g} with {len(self.axes)} Axes>")
import matplotlib.pyplot as plt

Preberimo datoteke

In [ ]:
netflix = pd.read_csv("Data\\netflix_titles.csv")

movies = pd.read_csv("Data\\akas.csv")
ratings = pd.read_csv("Data\\ratings.csv")
names = pd.read_csv("Data\\names_basic.csv")
crew=pd.read_csv("Data\\crew.csv")
crew.rename(columns = {'tconst':'titleId'}, inplace = True) 

director_names_and_stats=pd.read_csv("Data\\namebasics.csv")
director_names_and_stats.rename(columns = {'tconst':'titleId'}, inplace = True) 



In [ ]:
ratings = ratings.rename(columns = {'tconst':'titleId'})
originals = movies[movies["types"] == "original"]
originals = originals[["titleId", "title"]]
originals = originals.merge(ratings, on="titleId")
netflix1 = netflix.merge(originals, on="title")

projekt = originals.merge(crew[["titleId","directors"]],on="titleId")

In [ ]:
def najdi(table):
    if table.empty:
        return table
    else:
        return table.iloc[table["numVotes"].argmax()]

def getMovieDirID(MovieName):
    for index, row in projekt.iterrows():
        if (projekt.loc[index, row["title"]] == MovieName):
            return projekt.loc[index, row["directors"]]

def getDirector(dirID):
    df=director_names_and_stats[["nconst","primaryName"]]
    for index, row in df.iterrows():     
        if(df.loc[index, row["nconst"]] == dirID):
            return df.loc[index,"primaryName"]
            

In [ ]:
netflix1=netflix.fillna(0)

In [ ]:
i = 0
for idx, naslov in enumerate(netflix1["title"]):
    i += 1
    table = originals[originals["title"] == naslov]
    if table.empty:
        pass
    else:
        a = najdi(originals[originals["title"] == naslov])
        netflix1.loc[idx, "averageRating"] = a["averageRating"]
        netflix1.loc[idx, "numVotes"] = a["numVotes"]

Preberemo imdb bazo za mankajoce podatke

In [ ]:
from imdb import IMDb
from imdb import IMDbDataAccessError

movs = IMDb()
i = 0

for idx, naslov in enumerate(netflix1["title"]):
    i += 1
    if netflix1.loc[idx, "director"] == "0":
        m = movs.search_movie(naslov)
        if len(m) > 0:
            try:
                data = movs.get_movie(m[0].movieID)
                netflix1.loc[idx, "director"] = data["director"][0]['name']
            except KeyError as e:
                netflix1.loc[idx, "director"] = "-1"
            except (URLError, IMDbDataAccessErro) as e:
                print(data)
    if i%100 == 0:
        print(i)

Popravilo datumov

In [ ]:
dic = {'January':'1', 'February':'2', 'March':'3', 'April':'4', 'May':'5', 'June':'6', 'July':'7', 'August':'8', 'September':'9', 'October':'10', 'November':'11', 'December':'12', '':'0'}
netflix1['month_added'] = 0
netflix1['year_added'] = 0
for index, row in netflix1.iterrows():
    if(row['date_added'] != "0"):
        monthDay_year = row['date_added'].split(", ")
        day_month = monthDay_year[0].split(" ")
        if(len(day_month) > 2):
            netflix1.loc[index, "date_added"] = "%s-%s-%s" % (day_month[2], dic[day_month[1]],                                  monthDay_year[1])
            netflix1.loc[index, "month_added"]= dic[day_month[1]]
            netflix1.loc[index, "year_added"]= monthDay_year[1]
        else:
            netflix1.loc[index, "date_added"] = "%s-%s-%s" % (day_month[1], dic[day_month[0]],                                  monthDay_year[1])
            netflix1.loc[index, "month_added"]= dic[day_month[0]]
            netflix1.loc[index, "year_added"]= monthDay_year[1]

tu nevemo kaj nrdi

In [ ]:
#netflix1.to_csv("/Users/gregorsink/Desktop/PR Projekt/PR19KKZDBMPBPGS/Netflix_all.csv", index=False)
netflixContentByYears = netflix1.groupby(['year_added'])['year_added'].agg(['count'])
netflixContentByYears = netflixContentByYears.reset_index()
netflixContentByYears.drop([0])

#netflixContentByYears.set_index('index')

In [ ]:
#netflix1 = netflix1.sort_values(by='date_added', ascending=False)
#netflix1 = pd.to_datetime(netflix1['date_added'], dayfirst=True)
#netflix1['date_added'] = pd.to_datetime(netflix1['date_added']).dt.year
#netflix1['Year'] = pd.DatetimeIndex(netflix1['date_added']).month 
#tv_show = df[df["type"] == "TV Show"]
#movies = df[df["type"] == "Movie"]
for index, row in netflix1.iterrows():
    if(row['date_added'] != '0'):
        aaa = row['date_added']
        new = aaa.split("-")
        netflix1.loc[index, "month_added"]= new[1]
        netflix1[index, "year_added"]= new[2] 
        